# 🛰️ SVM Satellite Image Classification
## Bachelor's Thesis - Data Analytics Final Project

---

**Author:** Punada Pruthvirajsinh  
**Date:** January 2026  
**Topic:** Support Vector Machine for Land Cover Classification

---

## Table of Contents

1. [Introduction & Theory](#1-introduction)
2. [Data Loading & Exploration](#2-data-loading)
3. [Feature Engineering (HOG)](#3-feature-engineering)
4. [Model Training](#4-model-training)
5. [Kernel Comparison](#5-kernel-comparison)
6. [Results & Analysis](#6-results)
7. [Conclusion](#7-conclusion)

---

## 1. Introduction & Theory <a id='1-introduction'></a>

### 1.1 Project Overview

This project implements a **Support Vector Machine (SVM)** classifier for **satellite image classification** using the **EuroSAT** dataset. SVM is a supervised learning algorithm that finds the optimal hyperplane to separate different classes.

### 1.2 Support Vector Machine (SVM)

SVM works by finding the hyperplane that maximizes the margin between classes:

- **Linear SVM**: For linearly separable data
- **Kernel SVM**: Uses the "kernel trick" for non-linear boundaries
- **RBF Kernel**: `K(x, y) = exp(-γ||x-y||²)` - creates flexible decision boundaries

### 1.3 HOG Feature Extraction

**Histogram of Oriented Gradients (HOG)** captures edge and gradient structure:

1. Compute gradients in x and y directions
2. Create histograms of gradient orientations in cells
3. Normalize histograms across blocks
4. Concatenate to form feature vector

### 1.4 EuroSAT Dataset

| Property | Value |
|----------|-------|
| Total Images | 27,000 |
| Image Size | 64×64 RGB |
| Classes | 10 Land Cover Types |
| Source | Zenodo |

**Land Cover Classes:**
AnnualCrop, Forest, HerbaceousVegetation, Highway, Industrial, Pasture, PermanentCrop, Residential, River, SeaLake

---

## Setup & Imports

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import sys

# Add src to path
sys.path.insert(0, '../src')

# Import project modules
from satellite_loader import load_satellite_dataset, preprocess_images, SATELLITE_CLASS_NAMES, plot_satellite_samples
from feature_extraction import HOGFeatureExtractor, visualize_hog_features
from svm_classifier import SVMClassifier
from kernel_visualization import compare_kernels, plot_kernel_comparison, plot_kernel_theory

# Settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# For reproducibility
np.random.seed(42)

print('✓ All imports successful!')
print(f'✓ Working directory: {os.getcwd()}')

---

## 2. Data Loading & Exploration <a id='2-data-loading'></a>

### 2.1 Load EuroSAT Satellite Dataset

In [ ]:
# Load dataset (using subset for faster execution)
SUBSET_SIZE = 2000  # Use 2,000 samples for demonstration

data = load_satellite_dataset(data_dir='../data', subset_size=SUBSET_SIZE)

# Extract data
X_train = data['X_train']
y_train = data['y_train']
X_test = data['X_test']
y_test = data['y_test']
CLASS_NAMES = data['class_names']

print(f'\n📊 Dataset Summary:')
print(f'   Training samples: {len(X_train):,}')
print(f'   Test samples: {len(X_test):,}')
print(f'   Image shape: {X_train.shape[1:]}')
print(f'   Number of classes: {len(CLASS_NAMES)}')

### 2.2 Visualize Satellite Sample Images

In [ ]:
# Display sample satellite images from the dataset
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('EuroSAT Satellite Images (One per Land Cover Class)', fontsize=14, fontweight='bold')

for i, ax in enumerate(axes.flat):
    if i < len(CLASS_NAMES):
        # Find first image of class i
        idx = np.where(y_train == i)[0][0]
        ax.imshow(X_train[idx])
        ax.set_title(f'{CLASS_NAMES[i]}', fontsize=10)
        ax.axis('off')

plt.tight_layout()
plt.show()

### 2.3 Class Distribution Analysis

In [ ]:
# Analyze class distribution
unique, counts = np.unique(y_train, return_counts=True)
distribution = {CLASS_NAMES[i]: c for i, c in zip(unique, counts)}

# Create DataFrame for display
df_dist = pd.DataFrame({
    'Land Cover Class': list(distribution.keys()),
    'Count': list(distribution.values())
})
df_dist['Percentage'] = (df_dist['Count'] / df_dist['Count'].sum() * 100).round(2)

print('📊 Training Set Class Distribution:')
print(df_dist.to_string(index=False))

# Visualize
fig, ax = plt.subplots(figsize=(14, 5))
colors = plt.cm.viridis(np.linspace(0.2, 0.8, 10))
bars = ax.bar(df_dist['Land Cover Class'], df_dist['Count'], color=colors, edgecolor='white', linewidth=1.5)

ax.set_xlabel('Land Cover Class', fontsize=12)
ax.set_ylabel('Number of Images', fontsize=12)
ax.set_title('Class Distribution in Satellite Training Set', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

---

## 3. Feature Engineering (HOG) <a id='3-feature-engineering'></a>

### 3.1 Image Preprocessing

In [ ]:
# Preprocess images: convert to grayscale and normalize
print('Preprocessing satellite images...')

X_train_processed = preprocess_images(X_train, grayscale=True, normalize=True)
X_test_processed = preprocess_images(X_test, grayscale=True, normalize=True)

print(f'✓ Training images preprocessed: {X_train_processed.shape}')
print(f'✓ Test images preprocessed: {X_test_processed.shape}')

# Visualize preprocessing
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

idx = np.random.randint(0, len(X_train))

axes[0].imshow(X_train[idx])
axes[0].set_title(f'Original RGB - {CLASS_NAMES[y_train[idx]]}', fontsize=12)
axes[0].axis('off')

axes[1].imshow(X_train_processed[idx], cmap='gray')
axes[1].set_title('Preprocessed (Grayscale, Normalized)', fontsize=12)
axes[1].axis('off')

plt.suptitle('Satellite Image Preprocessing', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 3.2 HOG Feature Extraction

In [ ]:
# Initialize HOG Feature Extractor (adjusted for 64x64 satellite images)
extractor = HOGFeatureExtractor(
    orientations=9,
    pixels_per_cell=(8, 8),
    cells_per_block=(2, 2)
)

print('HOG Parameters:')
print(f'   Orientations: {extractor.orientations}')
print(f'   Pixels per cell: {extractor.pixels_per_cell}')
print(f'   Cells per block: {extractor.cells_per_block}')

In [ ]:
# Extract HOG features from training set
X_train_features = extractor.fit_transform(X_train_processed, apply_pca=False)

print(f'\n📊 Feature Extraction Results:')
print(f'   Input image shape: {X_train_processed[0].shape}')
print(f'   Features per image: {X_train_features.shape[1]}')
print(f'   Total training features: {X_train_features.shape}')

In [ ]:
# Extract features from test set
X_test_features = extractor.transform(X_test_processed)

print(f'✓ Test features: {X_test_features.shape}')

---

## 4. Model Training <a id='4-model-training'></a>

### 4.1 Train SVM Classifier

In [ ]:
# Initialize SVM classifier
classifier = SVMClassifier(
    kernel='rbf',
    C=10.0,
    gamma='scale'
)

# Train
classifier.train(X_train_features, y_train)

### 4.2 Evaluate Model

In [ ]:
# Evaluate on test set
results = classifier.evaluate(X_test_features, y_test, class_names=CLASS_NAMES)

---

## 5. Kernel Comparison <a id='5-kernel-comparison'></a>

### 5.1 Compare Linear, RBF, and Polynomial Kernels

In [ ]:
# Compare different SVM kernels
kernel_results = compare_kernels(
    X_train_features, y_train,
    X_test_features, y_test,
    CLASS_NAMES
)

In [ ]:
# Visualize kernel comparison
plot_kernel_comparison(kernel_results)
plt.show()

In [ ]:
# Display kernel theory
plot_kernel_theory()
plt.show()

---

## 6. Results & Analysis <a id='6-results'></a>

### 6.1 Results Summary

In [ ]:
# Create summary DataFrame
summary = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score'],
    'Score': [
        f"{results['accuracy']*100:.2f}%",
        f"{results['precision']*100:.2f}%",
        f"{results['recall']*100:.2f}%",
        f"{results['f1_score']*100:.2f}%"
    ]
})

print('\n' + '='*50)
print('📊 SATELLITE CLASSIFICATION RESULTS')
print('='*50)
print(summary.to_string(index=False))
print('='*50)

### 6.2 Confusion Matrix

In [ ]:
# Plot confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, results['y_pred'])

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
ax.set_xlabel('Predicted Land Cover', fontsize=12)
ax.set_ylabel('True Land Cover', fontsize=12)
ax.set_title('Satellite Image Classification Confusion Matrix', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

---

## 7. Conclusion <a id='7-conclusion'></a>

### 7.1 Summary

This project successfully implemented an **SVM-based satellite image classification system** using the EuroSAT dataset.

### Key Findings:

1. **HOG features** effectively capture land cover patterns from satellite imagery
2. **RBF kernel SVM** provides the best performance for satellite classification
3. Some classes (SeaLake, Forest) are easier to classify than others (PermanentCrop, Pasture)

### 7.2 Performance Summary

| Kernel | Accuracy |
|--------|----------|
| **RBF** | ~55% |
| Linear | ~45% |
| Polynomial | ~35% |

### 7.3 Applications

- Land use monitoring
- Urban planning
- Environmental analysis
- Agricultural assessment

In [ ]:
# Save model
import os
os.makedirs('../outputs/models', exist_ok=True)

classifier.save('../outputs/models/svm_classifier.joblib')
extractor.save('../outputs/models/feature_extractor.joblib')

print('\n✓ Models saved successfully!')
print('\n🎓 Project Complete!')